# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import input_file_name, regexp_extract, date_format, to_date
from awsglue.dynamicframe import DynamicFrame
## For getting dates from filenames


    
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 1
Setting new number of workers to: 2
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Session ID: 9a33027e-59ad-4c2d-8d2b-1d5efadd4f10
Applying the following default arguments:
--glue_kernel_version 1.0.4
--enable-glue-datacatalog true
Waiting for session 9a33027e-59ad-4c2d-8d2b-1d5efadd4f10 to get into ready status...
Session 9a33027e-59ad-4c2d-8d2b-1d5efadd4f10 has been created.



#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [2]:
input_dyf = glueContext.create_dynamic_frame.from_catalog(database='gpt-bucket-database', table_name='data', transformation_ctx='input_dyf')
input_dyf.printSchema()

root
|-- job_key: string
|-- location: string
|-- keyword: array
|    |-- element: string
|-- from_age: int
|-- page: int
|-- position: int
|-- salary_min: double
|-- salary_max: double
|-- salary_type: string
|-- salary_estimated_flag: int
|-- job_description: string
|-- company: string
|-- job_title: string
|-- url: string
|-- split_jd: string
|-- id: string
|-- object: string
|-- created: int
|-- model: string
|-- choices: array
|    |-- element: struct
|    |    |-- index: int
|    |    |-- message: struct
|    |    |    |-- role: string
|    |    |    |-- content: array
|    |    |    |    |-- element: string
|    |    |-- logprobs: null
|    |    |-- finish_reason: string
|-- usage: struct
|    |-- prompt_tokens: int
|    |-- completion_tokens: int
|    |-- total_tokens: int
|-- system_fingerprint: string


#### Change dyf to df to add filenames more easily.

In [3]:
input_df = input_dyf.toDF()

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.


In [4]:
input_df_with_filename = input_df.withColumn("filename", input_file_name())

In [5]:
input_df_with_filename.show(5)

+----------------+--------+--------------------+--------+----+--------+----------+----------+-----------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+----------+------------------+--------------------+--------------+------------------+--------------------+
|         job_key|location|             keyword|from_age|page|position|salary_min|salary_max|salary_type|salary_estimated_flag|     job_description|             company|           job_title|                 url|            split_jd|                  id|         object|   created|             model|             choices|         usage|system_fingerprint|            filename|
+----------------+--------+--------------------+--------+----+--------+----------+----------+-----------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------

#### Convert back to dyf for relationalization

In [6]:
dyf = DynamicFrame.fromDF(input_df_with_filename, glueContext, "dyf")
dyf.printSchema()
# dyf.show(5)

root
|-- job_key: string
|-- location: string
|-- keyword: array
|    |-- element: string
|-- from_age: int
|-- page: int
|-- position: int
|-- salary_min: double
|-- salary_max: double
|-- salary_type: string
|-- salary_estimated_flag: int
|-- job_description: string
|-- company: string
|-- job_title: string
|-- url: string
|-- split_jd: string
|-- id: string
|-- object: string
|-- created: int
|-- model: string
|-- choices: array
|    |-- element: struct
|    |    |-- index: int
|    |    |-- message: struct
|    |    |    |-- role: string
|    |    |    |-- content: array
|    |    |    |    |-- element: string
|    |    |-- logprobs: null
|    |    |-- finish_reason: string
|-- usage: struct
|    |-- prompt_tokens: int
|    |-- completion_tokens: int
|    |-- total_tokens: int
|-- system_fingerprint: string
|-- filename: string


## Relationalize (unnest) the dynamic frame and view the dyf pieces

In [7]:
unnested = dyf.relationalize('root', 's3://gpt-bucket-indeed/temp/')
unnested.keys()

dict_keys(['root', 'root_choices.val.message.content', 'root_keyword', 'root_choices'])


In [8]:
unnested.select('root_choices').toDF().show()

+---+-----+-----------------+------------------------+---------------------------+-------------------------+
| id|index|choices.val.index|choices.val.message.role|choices.val.message.content|choices.val.finish_reason|
+---+-----+-----------------+------------------------+---------------------------+-------------------------+
|  1|    0|                0|               assistant|                          1|                     stop|
|  2|    0|                0|               assistant|                          2|                     stop|
|  3|    0|                0|               assistant|                          3|                     stop|
|  4|    0|                0|               assistant|                          4|                     stop|
|  5|    0|                0|               assistant|                          5|                     stop|
|  6|    0|                0|               assistant|                          6|                     stop|
|  7|    0|        

In [9]:
unnested.select('root').toDF().show()

+----------------+--------+-------+--------+----+--------+----------+----------+-----------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+----------+------------------+-------+-------------------+-----------------------+------------------+------------------+--------------------+
|         job_key|location|keyword|from_age|page|position|salary_min|salary_max|salary_type|salary_estimated_flag|     job_description|             company|           job_title|                 url|            split_jd|                  id|         object|   created|             model|choices|usage.prompt_tokens|usage.completion_tokens|usage.total_tokens|system_fingerprint|            filename|
+----------------+--------+-------+--------+----+--------+----------+----------+-----------+---------------------+--------------------+--------------------+--------------------+--------------------+------

In [10]:
unnested.select('root_keyword').toDF().show()

+---+-----+--------------+
| id|index|   keyword.val|
+---+-----+--------------+
|  1|    0|  data analyst|
|  1|    1|data scientist|
|  2|    0|  data analyst|
|  3|    0|  data analyst|
|  4|    0|  data analyst|
|  5|    0|  data analyst|
|  6|    0|data scientist|
|  6|    1|  data analyst|
|  7|    0|  data analyst|
|  8|    0|data scientist|
|  9|    0|  data analyst|
|  9|    1|data scientist|
| 10|    0|  data analyst|
| 11|    0|  data analyst|
| 12|    0|  data analyst|
| 13|    0|  data analyst|
| 14|    0| data engineer|
| 15|    0|  data analyst|
| 16|    0|data scientist|
| 17|    0|  data analyst|
+---+-----+--------------+
only showing top 20 rows


In [11]:
unnested.select('root_choices.val.message.content').toDF().show()

+---+-----+-------------------------------+
| id|index|choices.val.message.content.val|
+---+-----+-------------------------------+
|  1|    0|             microsoft power bi|
|  1|    1|             nextgen ehr system|
|  2|    0|                            sql|
|  2|    1|                       power bi|
|  2|    2|                           domo|
|  2|    3|                        tableau|
|  2|    4|                         python|
|  2|    5|                              r|
|  3|    0|               google analytics|
|  3|    1|             google tag manager|
|  3|    2|                          excel|
|  3|    3|                  google sheets|
|  3|    4|             google data studio|
|  3|    5|             sisense data cloud|
|  4|    0|                              R|
|  4|    1|                            SQL|
|  4|    2|                         Python|
|  4|    3|                        Tableau|
|  5|    0|               google analytics|
|  5|    1|             google t

## Join Create DFs from DYFcollection

In [12]:
dyf_root = unnested.select('root')
dyf_root_keyword_full = unnested.select('root_keyword')
dyf_root_content = unnested.select('root_choices.val.message.content')

In [13]:
# Rename the content to techs
dyf_root_tech = dyf_root_content.rename_field(
    oldName="`choices.val.message.content.val`", # Requires `` around the name as it contains .'s
    newName='tech')
# dyf_root_tech.toDF().show(5)

# Rename the keywords to remove the period
dyf_root_keyword = dyf_root_keyword_full.rename_field(
    oldName='`keyword.val`',
    newName='keyword_val')
# dyf_root_keyword.toDF().show(5)

In [14]:
# Convert dyfs to dfs as we have a fixed schema with small memory usage so there aren't huge advantages to dyfs.
df_root = dyf_root.toDF()
df_root_keyword = dyf_root_keyword.toDF()
df_root_tech = dyf_root_tech.toDF()

In [15]:
df_root.printSchema()

root
 |-- job_key: string (nullable = true)
 |-- location: string (nullable = true)
 |-- keyword: long (nullable = true)
 |-- from_age: integer (nullable = true)
 |-- page: integer (nullable = true)
 |-- position: integer (nullable = true)
 |-- salary_min: double (nullable = true)
 |-- salary_max: double (nullable = true)
 |-- salary_type: string (nullable = true)
 |-- salary_estimated_flag: integer (nullable = true)
 |-- job_description: string (nullable = true)
 |-- company: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- split_jd: string (nullable = true)
 |-- id: string (nullable = true)
 |-- object: string (nullable = true)
 |-- created: integer (nullable = true)
 |-- model: string (nullable = true)
 |-- choices: long (nullable = true)
 |-- usage.prompt_tokens: integer (nullable = true)
 |-- usage.completion_tokens: integer (nullable = true)
 |-- usage.total_tokens: integer (nullable = true)
 |-- system_fingerprint: string 

In [16]:
df_root_selected = df_root.select('job_key', 'keyword', 'choices', 'filename')

In [17]:
df_root_selected.show(5)

+----------------+-------+-------+--------------------+
|         job_key|keyword|choices|            filename|
+----------------+-------+-------+--------------------+
|c40ac9eaa1278fdc|      1|      1|s3://gpt-bucket-i...|
|9da2149f4c0c676f|      2|      2|s3://gpt-bucket-i...|
|c7c487473ba0a838|      3|      3|s3://gpt-bucket-i...|
|62be243c0db32d5b|      4|      4|s3://gpt-bucket-i...|
|def7c0970b48a38e|      5|      5|s3://gpt-bucket-i...|
+----------------+-------+-------+--------------------+
only showing top 5 rows


In [18]:
df_root_keyword.show(5)

+---+-----+--------------+
| id|index|   keyword_val|
+---+-----+--------------+
|  1|    0|  data analyst|
|  1|    1|data scientist|
|  2|    0|  data analyst|
|  3|    0|  data analyst|
|  4|    0|  data analyst|
+---+-----+--------------+
only showing top 5 rows


In [19]:
join_root_keyword = df_root_selected.join(df_root_keyword, df_root['keyword'] == df_root_keyword['id'])

In [20]:
join_root_keyword = join_root_keyword \
                    .withColumnRenamed('index', 'keyword_index')

In [21]:
join_root_keyword.show(5)

+----------------+-------+-------+--------------------+---+-------------+--------------+
|         job_key|keyword|choices|            filename| id|keyword_index|   keyword_val|
+----------------+-------+-------+--------------------+---+-------------+--------------+
|9da2149f4c0c676f|      2|      2|s3://gpt-bucket-i...|  2|            0|  data analyst|
|62be243c0db32d5b|      4|      4|s3://gpt-bucket-i...|  4|            0|  data analyst|
|def7c0970b48a38e|      5|      5|s3://gpt-bucket-i...|  5|            0|  data analyst|
|7039cf33dfe411f5|      8|      8|s3://gpt-bucket-i...|  8|            0|data scientist|
|94fff78eb0d787f1|     12|     12|s3://gpt-bucket-i...| 12|            0|  data analyst|
+----------------+-------+-------+--------------------+---+-------------+--------------+
only showing top 5 rows


In [22]:
join_rk_techs = join_root_keyword.join(df_root_tech, on='id')

In [23]:
join_rk_techs = join_rk_techs \
                    .withColumnRenamed('index', 'tech_index')

In [24]:
join_rk_techs.show(5)

+---+----------------+-------+-------+--------------------+-------------+------------+----------+--------+
| id|         job_key|keyword|choices|            filename|keyword_index| keyword_val|tech_index|    tech|
+---+----------------+-------+-------+--------------------+-------------+------------+----------+--------+
|  2|9da2149f4c0c676f|      2|      2|s3://gpt-bucket-i...|            0|data analyst|         5|       r|
|  2|9da2149f4c0c676f|      2|      2|s3://gpt-bucket-i...|            0|data analyst|         4|  python|
|  2|9da2149f4c0c676f|      2|      2|s3://gpt-bucket-i...|            0|data analyst|         3| tableau|
|  2|9da2149f4c0c676f|      2|      2|s3://gpt-bucket-i...|            0|data analyst|         2|    domo|
|  2|9da2149f4c0c676f|      2|      2|s3://gpt-bucket-i...|            0|data analyst|         1|power bi|
+---+----------------+-------+-------+--------------------+-------------+------------+----------+--------+
only showing top 5 rows


In [25]:
cols_to_drop = ['keyword', 'choices']

In [26]:
join_dropped = join_rk_techs.drop(*cols_to_drop)

In [27]:
join_dropped.show(5)

+---+----------------+--------------------+-------------+------------+----------+--------+
| id|         job_key|            filename|keyword_index| keyword_val|tech_index|    tech|
+---+----------------+--------------------+-------------+------------+----------+--------+
|  2|9da2149f4c0c676f|s3://gpt-bucket-i...|            0|data analyst|         5|       r|
|  2|9da2149f4c0c676f|s3://gpt-bucket-i...|            0|data analyst|         4|  python|
|  2|9da2149f4c0c676f|s3://gpt-bucket-i...|            0|data analyst|         3| tableau|
|  2|9da2149f4c0c676f|s3://gpt-bucket-i...|            0|data analyst|         2|    domo|
|  2|9da2149f4c0c676f|s3://gpt-bucket-i...|            0|data analyst|         1|power bi|
+---+----------------+--------------------+-------------+------------+----------+--------+
only showing top 5 rows


In [10]:
# join_dropped.filter(join_dropped['job_key'] == '861f79f33ee60940').show(25)

#### Get the day/month/year from filename, change datatype of date column

In [52]:
df = join_dropped.withColumn("date", to_date(regexp_extract(join_dropped["filename"], r"\d{2}_\d{2}_\d{4}", 0), "dd_MM_yyyy"))

In [53]:
df.show(5)

+---+----------------+--------------------+-------------+------------+----------+--------+----------+
| id|         job_key|            filename|keyword_index| keyword_val|tech_index|    tech|      date|
+---+----------------+--------------------+-------------+------------+----------+--------+----------+
|  2|9da2149f4c0c676f|s3://gpt-bucket-i...|            0|data analyst|         5|       r|2024-03-02|
|  2|9da2149f4c0c676f|s3://gpt-bucket-i...|            0|data analyst|         4|  python|2024-03-02|
|  2|9da2149f4c0c676f|s3://gpt-bucket-i...|            0|data analyst|         3| tableau|2024-03-02|
|  2|9da2149f4c0c676f|s3://gpt-bucket-i...|            0|data analyst|         2|    domo|2024-03-02|
|  2|9da2149f4c0c676f|s3://gpt-bucket-i...|            0|data analyst|         1|power bi|2024-03-02|
+---+----------------+--------------------+-------------+------------+----------+--------+----------+
only showing top 5 rows


In [49]:
# Wasn't saving as a date type when in non- yyyy-MM-dd format.  Can look in to later, doesn't matter for now.
# df = df.withColumn("date", date_format("date", "dd-MM-yyyy"))

In [105]:
# df = join_dropped.withColumn("day", regexp_extract(join_dropped["filename"], r"\d{2}(?=_\d{2}_\d{4})", 0))
# df = df.withColumn("month", regexp_extract(df["filename"], r"(?<=_\d{2}_)\d{2}(?=_\d{4})", 0))
# df = df.withColumn("year", regexp_extract(df["filename"], r"\d{4}(?=.json$)", 0))

In [9]:
# df = df.withColumn("date", concat_ws("-", df["day"], df["month"], df["year"]))

In [55]:
final_df = df.drop('id', 'filename', 'keyword_index', 'tech_index')
final_df.show(5)

+----------------+------------+--------+----------+
|         job_key| keyword_val|    tech|      date|
+----------------+------------+--------+----------+
|9da2149f4c0c676f|data analyst|       r|2024-03-02|
|9da2149f4c0c676f|data analyst|  python|2024-03-02|
|9da2149f4c0c676f|data analyst| tableau|2024-03-02|
|9da2149f4c0c676f|data analyst|    domo|2024-03-02|
|9da2149f4c0c676f|data analyst|power bi|2024-03-02|
+----------------+------------+--------+----------+
only showing top 5 rows


In [56]:
final_df.printSchema()

root
 |-- job_key: string (nullable = true)
 |-- keyword_val: string (nullable = true)
 |-- tech: string (nullable = true)
 |-- date: date (nullable = true)


### Can see above that join_rk_techs is properly formatted. The job key had 12 techs associated and 2 keyword vals, so 24 lines in total with the techs repeating for each keyword val.
So in:
df_root 'keyword' is the 'id' in df_root_keyword
df_root 'choices' is the 'id' in df_root_tech

#### Example: Write the data in the DynamicFrame to a location in Amazon S3 and a table for it in the AWS Glue Data Catalog


In [ ]:
s3output = glueContext.getSink(
  path="s3://glue-bucket-indeed/processed",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=["date"],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="gpt-bucket-database", catalogTableName="results"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(DyF)